In [1]:
import h5py
import imageio
import numpy as np
import pandas as pd
import os
import datetime

from collections import defaultdict
from random import sample

In [2]:
np.random.seed(42)

In [3]:
hypercube_key = ['FOV2__1L','FOV3__1L','FOV4__1L','FOV2__1N','FOV3__1N',
                 'FOV2__2L','FOV3__2L','FOV4a__2L','FOV4b__2L','FOV5__2L','FOV2__2N','FOV3__2N',
                 'FOV4__2N','FOV5__2N','FOV2__3L','FOV3__3L','FOV4__3L','FOV2__3N','FOV3__3N',
                 'FOV4__3N','FOV5__3N','FOV6__3N','FOV2__4L','FOV3__4L','FOV4__4L','FOV5__4L',
                 'FOV6__4L','FOV7__4L','FOV8__4L','FOV9__4L','FOV2__4N','FOV3__4N','FOV4__4N',
                 'FOV5__4N','FOV6__4N','FOV7__4N','FOV2__5L','FOV3__5L','FOV4__5L','FOV5__5L',
                 'FOV6__5L','FOV7__5L','FOV8__5L','FOV9__5L','FOV10__5L','FOV2__5N','FOV3__5N',
                 'FOV4__5N','FOV5__5N','FOV6__5N','FOV7__5N','FOV8__5N','FOV9__5N','FOV10__5N',
                 'FOV2__6L','FOV3__6L','FOV4__6L','FOV5__6L','FOV2__6N','FOV3__6N','FOV4__6N',
                 'FOV5__6N','FOV6__6N','FOV7__6N','FOV2__7L','FOV3__7L','FOV4a__7L','FOV4b__7L', 'FOV5__7L',
                 'FOV6__7L','FOV7__7L','FOV8__7L','FOV9__7L','FOV10__7L','FOV11__7L','FOV12__7L',
                 'FOV13__7L','FOV14__7L','FOV15__7L','FOV2__7N','FOV3__7N','FOV4__7N','FOV5a__7N',
                 'FOV5b__7N','FOV6__7N','FOV2__8L','FOV3__8L','FOV4__8L','FOV5__8L','FOV6__8L',
                 'FOV7__8L','FOV8__8L','FOV9__8L','FOV10__8L','FOV2__8N','FOV3__8N','FOV4__8N',
                 'FOV5__8N','FOV6__8N','FOV7__8N','FOV2__9L','FOV3__9L','FOV4__9L','FOV5__9L',
                 'FOV6__9L','FOV7__9L','FOV8__9L','FOV2__9N','FOV3__9N','FOV4__9N','FOV5__9N',
                 'FOV6__9N','FOV7__9N','FOV8__9N','FOV9__9N','FOV2__10L','FOV3__10L','FOV4__10L',
                 'FOV5__10L','FOV6__10L','FOV7__10L','FOV8__10L','FOV9__10L','FOV2__10N','FOV3__10N',
                 'FOV4__10N','FOV5__10N','FOV6__10N','FOV7__10N','FOV8__10N','FOV9__10N','FOV10__10N',
                 'FOV11__10N','FOV12__10N','FOV13__10N','FOV2__12L','FOV3__12L','FOV4__12L','FOV5__12L',
                 'FOV6__12L','FOV7__12L','FOV8__12L','FOV9__12L','FOV10__12L','FOV11__12L','FOV2__12N',
                 'FOV3__12N','FOV4__12N','FOV5__12N','FOV6__12N','FOV7__12N','FOV8__12N','FOV9__12N','FOV10__12N',
                 'FOV2__13L','FOV3__13L','FOV4__13L','FOV5__13L','FOV6__13L','FOV2__13N','FOV3__13N','FOV4__13N',
                 'FOV5__13N','FOV6__13N','FOV7__13N','FOV4__14L','FOV5__14L','FOV2__14N','FOV3__14N','FOV4__14N',
                 'FOV5__14N','FOV6__14N','FOV7__14N','FOV8__14N','FOV9__14N']

In [4]:
patient_ID = np.array(['1','2','3','4','5','6','7','8','9','10','12','13','14'])
patient_images = np.array(['5','9','8','14','18','10','21','15','15','20','19','11', '10'])
patient_index = np.c_[patient_ID.reshape(len(patient_ID), -1), patient_images.reshape(len(patient_images), -1)]
print(patient_index)

[['1' '5']
 ['2' '9']
 ['3' '8']
 ['4' '14']
 ['5' '18']
 ['6' '10']
 ['7' '21']
 ['8' '15']
 ['9' '15']
 ['10' '20']
 ['12' '19']
 ['13' '11']
 ['14' '10']]


In [5]:
np.random.shuffle(patient_index)
print(patient_index)

[['13' '11']
 ['10' '20']
 ['1' '5']
 ['9' '15']
 ['6' '10']
 ['3' '8']
 ['2' '9']
 ['14' '10']
 ['5' '18']
 ['8' '15']
 ['12' '19']
 ['4' '14']
 ['7' '21']]


In [6]:
test_ratio = 0.2
test_images = np.ceil(test_ratio*len(hypercube_key))
print(test_images)

35.0


In [7]:
hypercube_key_sorted = ['FOV2__13L','FOV3__13L','FOV4__13L','FOV5__13L','FOV6__13L','FOV2__13N','FOV3__13N',
                        'FOV4__13N','FOV5__13N','FOV6__13N','FOV7__13N', 'FOV2__10L','FOV3__10L','FOV4__10L',
                        'FOV5__10L','FOV6__10L','FOV7__10L','FOV8__10L','FOV9__10L','FOV2__10N','FOV3__10N',
                        'FOV4__10N','FOV5__10N','FOV6__10N','FOV7__10N','FOV8__10N','FOV9__10N','FOV10__10N',
                        'FOV11__10N','FOV12__10N','FOV13__10N','FOV2__1L','FOV3__1L','FOV4__1L','FOV2__1N',
                        'FOV3__1N','FOV2__9L','FOV3__9L','FOV4__9L','FOV5__9L','FOV6__9L','FOV7__9L','FOV8__9L',
                        'FOV2__9N','FOV3__9N','FOV4__9N','FOV5__9N','FOV6__9N','FOV7__9N','FOV8__9N','FOV9__9N',
                        'FOV2__6L','FOV3__6L','FOV4__6L','FOV5__6L','FOV2__6N','FOV3__6N','FOV4__6N','FOV5__6N',
                        'FOV6__6N','FOV7__6N', 'FOV2__3L','FOV3__3L','FOV4__3L','FOV2__3N','FOV3__3N','FOV4__3N',
                        'FOV5__3N','FOV6__3N','FOV2__2L','FOV3__2L','FOV4a__2L','FOV4b__2L','FOV5__2L','FOV2__2N',
                        'FOV3__2N','FOV4__2N','FOV5__2N','FOV4__14L','FOV5__14L','FOV2__14N','FOV3__14N','FOV4__14N',
                        'FOV5__14N','FOV6__14N','FOV7__14N','FOV8__14N','FOV9__14N','FOV2__5L','FOV3__5L','FOV4__5L',
                        'FOV5__5L','FOV6__5L','FOV7__5L','FOV8__5L','FOV9__5L','FOV10__5L','FOV2__5N','FOV3__5N',
                        'FOV4__5N','FOV5__5N','FOV6__5N','FOV7__5N','FOV8__5N','FOV9__5N','FOV10__5N','FOV2__8L',
                        'FOV3__8L','FOV4__8L','FOV5__8L','FOV6__8L','FOV7__8L','FOV8__8L','FOV9__8L','FOV10__8L',
                        'FOV2__8N','FOV3__8N','FOV4__8N','FOV5__8N','FOV6__8N','FOV7__8N','FOV2__12L','FOV3__12L',
                        'FOV4__12L','FOV5__12L','FOV6__12L','FOV7__12L','FOV8__12L','FOV9__12L','FOV10__12L','FOV11__12L',
                        'FOV2__12N','FOV3__12N','FOV4__12N','FOV5__12N','FOV6__12N','FOV7__12N','FOV8__12N','FOV9__12N',
                        'FOV10__12N','FOV2__4L','FOV3__4L','FOV4__4L','FOV5__4L','FOV6__4L','FOV7__4L','FOV8__4L',
                        'FOV9__4L','FOV2__4N','FOV3__4N','FOV4__4N','FOV5__4N','FOV6__4N','FOV7__4N','FOV2__7L','FOV3__7L',
                        'FOV4a__7L','FOV4b__7L', 'FOV5__7L','FOV6__7L','FOV7__7L','FOV8__7L','FOV9__7L','FOV10__7L',
                        'FOV11__7L','FOV12__7L','FOV13__7L','FOV14__7L','FOV15__7L','FOV2__7N','FOV3__7N','FOV4__7N',
                        'FOV5a__7N','FOV5b__7N','FOV6__7N']

In [8]:
print('Total Hypercubes: ', len(hypercube_key_sorted))

Total Hypercubes:  175


In [9]:
print(hypercube_key_sorted)

['FOV2__13L', 'FOV3__13L', 'FOV4__13L', 'FOV5__13L', 'FOV6__13L', 'FOV2__13N', 'FOV3__13N', 'FOV4__13N', 'FOV5__13N', 'FOV6__13N', 'FOV7__13N', 'FOV2__10L', 'FOV3__10L', 'FOV4__10L', 'FOV5__10L', 'FOV6__10L', 'FOV7__10L', 'FOV8__10L', 'FOV9__10L', 'FOV2__10N', 'FOV3__10N', 'FOV4__10N', 'FOV5__10N', 'FOV6__10N', 'FOV7__10N', 'FOV8__10N', 'FOV9__10N', 'FOV10__10N', 'FOV11__10N', 'FOV12__10N', 'FOV13__10N', 'FOV2__1L', 'FOV3__1L', 'FOV4__1L', 'FOV2__1N', 'FOV3__1N', 'FOV2__9L', 'FOV3__9L', 'FOV4__9L', 'FOV5__9L', 'FOV6__9L', 'FOV7__9L', 'FOV8__9L', 'FOV2__9N', 'FOV3__9N', 'FOV4__9N', 'FOV5__9N', 'FOV6__9N', 'FOV7__9N', 'FOV8__9N', 'FOV9__9N', 'FOV2__6L', 'FOV3__6L', 'FOV4__6L', 'FOV5__6L', 'FOV2__6N', 'FOV3__6N', 'FOV4__6N', 'FOV5__6N', 'FOV6__6N', 'FOV7__6N', 'FOV2__3L', 'FOV3__3L', 'FOV4__3L', 'FOV2__3N', 'FOV3__3N', 'FOV4__3N', 'FOV5__3N', 'FOV6__3N', 'FOV2__2L', 'FOV3__2L', 'FOV4a__2L', 'FOV4b__2L', 'FOV5__2L', 'FOV2__2N', 'FOV3__2N', 'FOV4__2N', 'FOV5__2N', 'FOV4__14L', 'FOV5__14L', 

In [10]:
print('Total Segmented Images:\t', len(hypercube_key_sorted)*100)

Total Segmented Images:	 17500


In [11]:
bands_root_path = "Smaller Segmented Hypercubes"
bands_classes = sorted([dirname for dirname in os.listdir(bands_root_path)
                  if os.path.isdir(os.path.join(bands_root_path, dirname))])
bands_classes

['Lesional', 'Nonlesional']

In [12]:
image_paths = defaultdict(list)

for bands_class in bands_classes:
    image_dir = os.path.join(bands_root_path, bands_class)
    for filepath in os.listdir(image_dir):
        if filepath.endswith(".tif"):
            image_paths[bands_class].append(os.path.join(image_dir, filepath))

In [13]:
for paths in image_paths.values():
    paths.sort()

In [14]:
bands_class_ids = {bands_class: index for index, bands_class in enumerate(bands_classes)}
bands_class_ids

{'Lesional': 0, 'Nonlesional': 1}

In [15]:
bands_paths_and_classes_sorted = []
for hypercube in hypercube_key_sorted:
    for bands_class, paths in image_paths.items():
        for path in paths:
            if hypercube in path:
                bands_paths_and_classes_sorted.append((path, bands_class_ids[bands_class]))

In [16]:
bands_paths_and_classes_sorted[0]

('Smaller Segmented Hypercubes\\Lesional\\Lesional_50_10_10_FOV2__13L__multi_out.tif',
 0)

In [17]:
bands_paths_and_classes_sorted[1]

('Smaller Segmented Hypercubes\\Lesional\\Lesional_50_10_1_FOV2__13L__multi_out.tif',
 0)

In [18]:
fileName = 'Small_Segmented_Hypercubes_imageminmax_rseed42.hdf5'
SSHSamples = len(hypercube_key_sorted)*100
with h5py.File(fileName, "w") as hdf5_file:
    hdf5_file.create_dataset("X_SSH",(SSHSamples,95000),np.float32)
    hdf5_file.create_dataset("y_SSH",(SSHSamples,),np.int16)        

In [19]:
i=0
compiling_start_time = datetime.datetime.now()
print("Start Time: ", compiling_start_time, "\n")
with h5py.File(fileName, "a") as hdf5_file:
    for path, label in bands_paths_and_classes_sorted:
        img = imageio.volread(path)
        hypimage = np.asarray(img)
        hypimage = (hypimage-np.min(hypimage))/(np.max(hypimage)-np.min(hypimage))
        hypimage = hypimage.flatten()
        img_label = label
        hdf5_file["X_SSH"][i] = np.asarray(hypimage)
        hdf5_file["y_SSH"][i] = np.asarray(img_label)
        if i % 100 == 0 and i > 1:
            print('SSH data: {}/{}'.format(i, len(bands_paths_and_classes_sorted)))
        i=i+1
compiling_end_time = datetime.datetime.now()
compiling_duration = compiling_end_time-compiling_start_time
compiling_hours, compiling_remainder = divmod(compiling_duration.seconds, 3600)
compiling_minutes, compiling_seconds = divmod(compiling_remainder,60)
print('\nCompiling Complete:', compiling_end_time, '\n Total time taken:', compiling_hours," hours, ", compiling_minutes,
      " minutes, and ", compiling_seconds, " seconds.\n")

Start Time:  2019-03-14 21:23:00.471239 



C:\Users\005375049\AppData\Roaming\Python\Python35\site-packages\imageio\plugins\_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'decode_packbits'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
C:\Users\005375049\AppData\Roaming\Python\Python35\site-packages\imageio\plugins\_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'decode_lzw'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
C:\Users\005375049\AppData\Roaming\Python\Python35\site-packages\imageio\plugins\_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'unpack_ints'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
C:\Users\005375049\AppData\Roaming\Python\Python35\site-packages\imageio\plugins\_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'reverse_bitorder'
  Functionality might be de

SSH data: 100/17500
SSH data: 200/17500
SSH data: 300/17500
SSH data: 400/17500
SSH data: 500/17500
SSH data: 600/17500
SSH data: 700/17500
SSH data: 800/17500
SSH data: 900/17500
SSH data: 1000/17500
SSH data: 1100/17500
SSH data: 1200/17500
SSH data: 1300/17500
SSH data: 1400/17500
SSH data: 1500/17500
SSH data: 1600/17500
SSH data: 1700/17500
SSH data: 1800/17500
SSH data: 1900/17500
SSH data: 2000/17500
SSH data: 2100/17500
SSH data: 2200/17500
SSH data: 2300/17500
SSH data: 2400/17500
SSH data: 2500/17500
SSH data: 2600/17500
SSH data: 2700/17500
SSH data: 2800/17500
SSH data: 2900/17500
SSH data: 3000/17500
SSH data: 3100/17500
SSH data: 3200/17500
SSH data: 3300/17500
SSH data: 3400/17500
SSH data: 3500/17500
SSH data: 3600/17500
SSH data: 3700/17500
SSH data: 3800/17500
SSH data: 3900/17500
SSH data: 4000/17500
SSH data: 4100/17500
SSH data: 4200/17500
SSH data: 4300/17500
SSH data: 4400/17500
SSH data: 4500/17500
SSH data: 4600/17500
SSH data: 4700/17500
SSH data: 4800/17500
S

In [20]:
hdf5_file.close()